<center><b><font color='red' size=8>Impact of September 11 on Air Travel in the United States</font></b></center>

---
### An Introduction To The Problem. "Background"
#### The Research and Innovative Technology Administration's Bureau of Transportation Statistics conducted a study to assess the impact of the September 11, 2001, terrorist attack on transportation in the US. The study aimed to analyze passenger travel behaviour patterns before and after 9/11. The report and data from the 2006 study can be found at [here](https://www.bts.gov/archive/publications/estimated_impacts_of_9_11_on_us_travel/index).
---
### Data Analysis:
#### The study analyzed monthly passenger movement data between January 1990 and May 2004. The provided dataset, named Sept11Travel.csv, includes three monthly time series: actual airline revenue passenger miles (Air), rail passenger miles (Rail), and vehicle miles travelled (Car). To evaluate the impact of September 11, the Bureau of Transportation Statistics employed the following approach: they forecasted future data based on pre-September 11 information, assuming no terrorist attack, and compared the forecasted series with the actual data to determine the event's impact. Initially, the focus is on the pre-event time series.
#### `Tasks:` The methods within each task as an ideas from me, but some of the implementation by Chat-Gpt.
1. ####  Plot the pre-event AIR time series. What time series components appear? [Go to S.1](#S.1)
2. #### Figure 17.11 shows a time plot of the seasonally adjusted pre-September-11 AIR series. Which of the following methods would be adequate for forecasting the series shown in the figure? [Go to S.2](#S.2)
    * ##### Linear regression model seasonality
    * ##### Linear regression model with trend
    * ##### Linear regression model with trend and seasonality
![image.png](attachment:d2e1e9fd-6b86-412c-a290-225fe2752f19.png)
3. ####  Specify a linear regression model for the AIR series that would produce a seasonally adjusted series similar to the one shown in Figure 17.11, with multiplicative seasonality. What is the outcome variable? What are the predictors? [Go to S.3](#S.3)
4. ####  Run the regression model from (c). Remember to use only pre-event data. [Go to S.4](#S.4)
    * ##### What can we learn from the statistical insignificance of the coefficients for October and September?
    * ##### The actual value of AIR (air revenue passenger miles) in January 1990 was 35.153577 billion. What is the residual for this month, using the regression model? Report the residual in terms of air revenue passenger miles.
5. ####  Create an ACF (autocorrelation) plot of the regression residuals. [Go to S.5](#S.5)
    * ##### What does the ACF plot tell us about the regression model’s forecasts?
    * ##### How can this information be used to improve the model?
6. ####  Fit linear regression models to Air, Rail, and to Auto with additive seasonality and an appropriate trend. For Air and Rail, fit a linear trend. For Rail, use a quadratic trend. Remember to use only pre-event data. Once the models are estimated, use them to forecast each of the three post-event series. [Go to S.6](#S.6)
    * ##### For each series (Air, Rail, Auto), plot the complete pre-event and post-event actual series overlayed with the predicted series.
    * ##### What can be said about the effect of the September 11 terrorist attack on the three modes of transportation? Discuss the magnitude of the effect, its time span, and any other relevant aspects

---
* #### Import Necessary Modules and Packages For The Implementation, This May Include:
    - ##### Pandas.
    - ##### Numpy.
    - ##### Matplotlib.
    - ##### Seaborn.
    - ##### Statsmodels.m
    - ##### Sk-Learn.
    - ##### ....
---

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

---
* #### Read The Data and Have a General Look At It, To Gain More Insights About It.
    - ##### `Note:` We Will Skip Reading The First 2 Rows From The Data, Since They Are Just Headers, and The Actual Dataset Starts After The First 2 Rows.
---

In [ ]:
data = pd.read_csv('table_c01.csv', skiprows=2)
data

,Unnamed: 0,Air RPM (000s),Rail PM,VMT (billions)
0,Jan-90,"35,153,577","454,115,779",163.28
1,Feb-90,"32,965,187","435,086,002",153.25
2,Mar-90,"39,993,913","568,289,732",178.42
3,Apr-90,"37,981,886","568,101,697",178.68
4,May-90,"38,419,672","539,628,385",188.88
...,...,...,...,...
168,Jan-04,"53,447,972","410,338,691",217.30
169,Feb-04,"52,608,801","389,778,365",210.40
170,Mar-04,"63,600,019","453,014,590",247.50
171,Apr-04,"61,887,720","471,116,666",245.40


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      173 non-null    object 
 1   Air RPM (000s)  173 non-null    object 
 2   Rail PM         173 non-null    object 
 3   VMT (billions)  172 non-null    float64
dtypes: float64(1), object(3)
memory usage: 5.5+ KB


---
* #### We Can See That From The Previous Cells, That The First column Is Supposed To Represent The Date. Also We Got 1 Missing Value In `VMT (billions)` Column, and For The Data Types, All of Them Except `VMT (billions)` Column Needs To Be Converted Into The Right Dtype.
* #### We Are Going To Perform Some Simple Preprocessing, To Make The Data More Easy To Read and Correct For Many Sides.
---

In [ ]:
data.columns = ['Date','Air_RPM_000s', 'Rail_PM', 'VMT_billions']
data['Air_RPM_000s'] = pd.to_numeric(data['Air_RPM_000s'].str.replace(',', ''), errors='coerce')
data['Rail_PM'] = pd.to_numeric(data['Rail_PM'].str.replace(',', ''), errors='coerce')
data['Date'] = pd.to_datetime(data['Date'], format='%b-%y')
data.head()

,Date,Air_RPM_000s,Rail_PM,VMT_billions
0,1990-01-01,35153577,454115779,163.28
1,1990-02-01,32965187,435086002,153.25
2,1990-03-01,39993913,568289732,178.42
3,1990-04-01,37981886,568101697,178.68
4,1990-05-01,38419672,539628385,188.88


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          173 non-null    datetime64[ns]
 1   Air_RPM_000s  173 non-null    int64         
 2   Rail_PM       173 non-null    int64         
 3   VMT_billions  172 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 5.5 KB


---
<a id="S.1"></a>
* #### This Is The Solution For The First Question.
---

In [ ]:
preEvent_mask = data['Date'] <= pd.Timestamp('2001-09-11')
preEvent_data = data[preEvent_mask]
preEvent_data.head()

In [ ]:
# Plot the pre-event data
plt.plot(preEvent_data['Date'], preEvent_data['Air_RPM_000s'])
plt.xlabel('Date')
plt.ylabel('Air RPM (000s)')
plt.title('Pre-Event AIR Time Series')
plt.xticks(rotation=45)
plt.show()

---
* ### From The Graph Up, We Can See That 3 of Time Series Components Exists:
    - #### Noise: As "It is always present in a time series to some degree Pg442.", and It Represents The Random or Unpredictable Fluctuations in The Time Series That Cannot be Attributed To Other Components.
    - #### Trend: Since There Is a Clear and Persistent Upward or Downward Movement in The Time Series Plot.
    - #### Seasonality: Since There Is Repeated Patterns or Cycles That Occur Within a Fixed Time Interval
* ##### `Note:` Level component does not exist since there is no part of the time series plot where the values there is like constant through a period of time, to present a horizontal line without any upward or downward movement.

---
<a id="S.2"></a>
* ### This Is The Solution For The Second Question.
    - #### From The Graph in The Question, We Will Pick `Linear Regression Model With Trend`, Since There Is Only `Trend` Component Appear in The Plot. But We Might Assume That There Is `Seasonality`, Since There Is At Some Small Periods, We May Consider Them As Cycles.

---
<a id="S.3"></a>
* ### This Is The Solution For The Third Question.
* #### In This Part, We Will:
    - ##### Create Dummy Variables For Each Month Using `pd.get_dummies()` After Extracting Them From The Dataset, Which Will Serve As Predictors Capturing The Multiplicative Seasonality
    - ##### The `sm.add_constant()` Function Adds an Intercept Column To The Predictors.
    - ##### Fit The Linear Regression Model `OLS`.
    - ##### Predict To Obtain The Estimated Coefficients And Other Statistical Information.
    - ##### Print Out a Summary of The Model, Including The `Coefficients`, `p-values`, and Other Statistical Measures.
---

In [ ]:
preEvent_data['Month'] = preEvent_data['Date'].dt.month
preEvent_data.head()

In [ ]:
X = pd.get_dummies(preEvent_data['Month'], prefix='month')  # Predictors.
y = preEvent_data['Air_RPM_000s']  # Outcome Variable.

X = sm.add_constant(X)

model = sm.OLS(y, X)
results = model.fit()

seasonally_adjusted = results.predict(X)

# Print the model summary
print(results.summary())

---
* ### From The Summary of The Model Up, and The Approach We Used, We Can Determine That:
    - #### The Outcome Variable: 'Air RPM (000s)'
    - #### The Predictors: Dummy Variables Representing The Months, e.g., 'month_1', 'month_2', 'month_3', and so on.

---
<a id="S.4"></a>
* ### This Is The Solution For The Forth Question.
* ### In This Part, We Have:
    - #### Run The Regression Model From The Previous Question.
    - #### Determined What Can We Learn From The Statistical Insignificance of The Coefficients For October and September.
        + ##### `Answer:` From The Statistical Insignificance of The Coefficients For October and September, Indicated By The P-values Of 0.111 and 0.568 Respectively, We Can Conclude That There Is No Strong Evidence To Suggest A Significant Effect Of These Months On The `Air_RPM_000s` Series. Therefore, Based On The Current Model And Data, We Cannot Conclude That October Or September Have A Significant Impact On The Series.
    - #### Determined What The Residual Is For `Jan 1990` Month Using The Regression Model, and Report The Residual In Terms of Air Revenue Passenger Miles.
        + ##### `Answer:` Using The Regression Model, The Predicted Value For The `Air_RPM_000s` Series In January 1990 Is 44,400,000 (44.4 Million) Based On The Constant Coefficient (Intercept) of 4.44e+07. Therefore, The Residual Can Be Calculated As:
            * ###### Residual = Actual value - Predicted value = 35.153577 billion - 44,400,000.
            * ###### The resulting value represents the residual in terms of air revenue passenger miles for January 1990. Please ensure that the units of the actual value and the regression model predictions are consistent (e.g., both in billions or both in millions) to obtain an accurate residual.

---
<a id="S.5"></a>
* ### This Is The Solution For The Fifth Question.
* ### In This Part, We Have:
    - #### Determined What The `ACF` Plot Tells Us About The Regression Model’s Forecasts.
        + ##### `Answer:` The `ACF` Plot Helps Us Assess The Autocorrelation of The Residuals, Which Is A Measure of How The Residuals Are Correlated With Their Lagged Values. If The Acf Plot Shows That The Autocorrelation Values Are Close To Zero For Most Lags And Fall Within The Confidence Intervals (Represented By The Blue Shaded Area), It Indicates That The Regression Model's Forecasts Do Not Exhibit Significant Systematic Patterns or Correlation Structure.
    - #### Determined How This Information Can Be Used To Improve The Model.
        + ##### `Answer:` The information Obtained From The `ACF` Plot Can Be Used To Improve The Model In The Following Ways:
            1. ###### Identify Autocorrelation: The `ACF` Plot Helps Identify The Presence Of Autocorrelation In The Residuals of The Regression Model. If Significant Autocorrelation Is Observed At Certain Lags, It Indicates That The Current Model May Not Capture All The Temporal Dependencies In The Data.
            
            2. ###### Refine Predictor Variables: Based On The Autocorrelation Patterns Observed In The Acf Plot, We Can Refine The Predictor Variables Used In The Model. This May Involve Adding Lagged Values of The Target Variable or Incorporating Other Relevant Time-related Features To Capture The Autocorrelation Structure Better.
            
            3. ###### Explore Time Series Models: If The Autocorrelation Is Substantial and Cannot Be Adequately Captured By Refining The Predictor Variables In A Linear Regression Model, It May Be Necessary To Explore Time Series Models. Time Series Models, Such As Arima Or Sarimax, Are Specifically Designed To Handle Autocorrelation And Can Provide Better Forecasts By Considering The Temporal Dependencies In The Data.
            
            4. ###### Model Residuals: Analyzing The `ACF` Plot of The Residuals Can Reveal Any Remaining Patterns or Correlations That Are Not Captured By The Current Model. By Incorporating Additional Terms or More Sophisticated Modeling Techniques, Such As Autoregressive or Moving Average Terms, You Can Account For The Residual Autocorrelation And Further Improve The Model's Forecasting Performance.

In [ ]:
# Calculate the residuals
residuals = results.resid
# Assuming 'residuals' is the variable that contains the regression residuals
plot_acf(residuals, lags=20)  # Adjust the number of lags as needed
plt.xlabel('Lags')
plt.ylabel('Autocorrelation')
plt.title('Autocorrelation Plot of Regression Residuals')
plt.show()

---
<a id="S.6"></a>
* ### This Is The Solution For The Sixth `Final` Question.
* ### In This Part, We Have:
    - #### Got Data After That Event.
    - #### Fitted Linear Regression Mdel To respected Columns With Additive Seasonality and Linear Trend `SARIMAX`.
    - #### Estimated The Models.
    - #### Forecasted Each of The Three `post-event` Series Using The Estimatted Models.
    - #### Stored The Forecasted Values Into `post-event` DataFrame.
    - #### Plotted The `pre-event` Actual `Air RPM, Rail PM, VMT` Values Against The Corresponding Dates.
    - #### Plotted The `post-event` Actual `Air RPM, Rail PM, VMT` Values Against The Corresponding Dates.
    - #### Determined What Can Be Said About The Effect Of The September 11 Terrorist Attack on Thethree Modes of Transportation, The Magnitude of The Effect, Its Time Span, ....
        + ##### `Answer:` Based On The Observations From The Graps Below, It Can Be Concluded That The September 11 Attack Had A Notable Effect on `Air Transportation`, With a Visible Drop In The Post-event Period. However, `Rail` and `VMT` Do Not Exhibit Significant Changes In The Post-event Series.

In [ ]:
postEvent_mask = data['Date'] > pd.Timestamp('2001-09-11')
postEvent_data = data[postEvent_mask]
postEvent_data.head()

In [ ]:
air_model = sm.tsa.statespace.SARIMAX(preEvent_data['Air_RPM_000s'], order=(1,1,1), seasonal_order=(1,1,1,12), trend='t')
rail_model = sm.tsa.statespace.SARIMAX(preEvent_data['Rail_PM'], order=(1,1,1), seasonal_order=(1,1,1,12), trend='t', trend_offset=1)
vmt_model = sm.tsa.statespace.SARIMAX(preEvent_data['VMT_billions'], order=(1,1,1), seasonal_order=(1,1,1,12), trend='t')

air_results = air_model.fit()
rail_results = rail_model.fit()
auto_results = vmt_model.fit()

air_forecast = air_results.forecast(steps=len(postEvent_data))
rail_forecast = rail_results.forecast(steps=len(postEvent_data))
auto_forecast = auto_results.forecast(steps=len(postEvent_data))

postEvent_data['Predicted_Air'] = air_forecast
postEvent_data['Predicted_Rail'] = rail_forecast
postEvent_data['Predicted_VMT'] = auto_forecast

postEvent_data.head()

In [ ]:
plt.plot(preEvent_data['Date'], preEvent_data['Air_RPM_000s'], label='Pre-event Actual')
plt.plot(postEvent_data['Date'], postEvent_data['Air_RPM_000s'], label='Post-event Actual')
plt.plot(postEvent_data['Date'], postEvent_data['Predicted_Air'], label='Predicted')
plt.xlabel('Date')
plt.ylabel('Air Transportation (millions)')
plt.title('Air Transportation')
plt.legend()
plt.show()

plt.plot(preEvent_data['Date'], preEvent_data['Rail_PM'], label='Pre-event Actual')
plt.plot(postEvent_data['Date'], postEvent_data['Rail_PM'], label='Post-event Actual')
plt.plot(postEvent_data['Date'], postEvent_data['Predicted_Rail'], label='Predicted')
plt.xlabel('Date')
plt.ylabel('Rail PM')
plt.title('Rail PM')
plt.legend()
plt.show()

plt.plot(preEvent_data['Date'], preEvent_data['VMT_billions'], label='Pre-event Actual')
plt.plot(postEvent_data['Date'], postEvent_data['VMT_billions'], label='Post-event Actual')
plt.plot(postEvent_data['Date'], postEvent_data['Predicted_VMT'], label='Predicted')
plt.xlabel('Date')
plt.ylabel('VMT (billions)')
plt.title('VMT (billions)')
plt.legend()
plt.show()